In [1]:
import pickle as pkl
import numpy as np
import keras
import boto3
import numpy as np
import pandas as pd
import os
import json
from datetime import timedelta
from dateutil import parser
import boto3
import urllib.request
import datetime
import sklearn
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Using TensorFlow backend.
C:\Users\eric_\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\eric_\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\eric_\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\eric_\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data"
X = pkl.load(open("x_master.p", "rb"))
Y = pkl.load(open("y_master.p", "rb"))

X.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data


(90487,)

In [3]:
x_bin=[]
y_bin=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(Y):
  if X[i].shape == (160, 4):
      if y in [1,2,3]:
        y_bin.append(1)
        x_bin.append(X[i])
        smoking_event+=1
      elif y >= 20:
        y_bin.append(0)
        x_bin.append(X[i])
        non_smoking_event+=1
        
x_bin=np.array(x_bin)
y_bin=np.array(y_bin)

In [4]:
x_bin.shape

(83369, 160, 4)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x_bin, y_bin, test_size=0.1, random_state = 99)
x_train.shape

(75032, 160, 4)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv1D, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
from keras import regularizers

In [9]:
print(len(y_train))
np.count_nonzero(y_train)

75032


2485

In [ ]:
early_stop=EarlyStopping(monitor='val_loss', min_delta=.0001, patience=10, verbose=0, mode='auto',
                         baseline=None, restore_best_weights=False)
class_weight = {0: 1, 1: 30}

model = Sequential()
model.add(Conv1D(filters=64 ,kernel_size=2,kernel_initializer = 'he_normal' ,activation='relu', 
                 input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.1))
model.add(Conv1D(filters=64 ,kernel_size=2,kernel_initializer = 'he_normal', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(.1))
model.add(Flatten())
model.add(Dense(64,kernel_initializer = 'he_normal', activation= 'elu'))
model.add(Dense(1 ,kernel_initializer = 'he_normal', activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
history = model.fit(x_bin, y_bin, batch_size=32,epochs=500,validation_split = .1, callbacks = [early_stop], 
                   class_weight = class_weight)
#model.evaluate(x_test, y_test, verbose=0)

Train on 75032 samples, validate on 8337 samples
Epoch 1/500
75032/75032 [==============================] - 55s 736us/sample - loss: 0.6164 - acc: 0.8551 - val_loss: 0.3144 - val_acc: 0.8827
Epoch 2/500
75032/75032 [==============================] - 52s 695us/sample - loss: 0.4412 - acc: 0.8968 - val_loss: 0.3575 - val_acc: 0.8625
Epoch 3/500
75032/75032 [==============================] - 54s 714us/sample - loss: 0.3755 - acc: 0.9142 - val_loss: 0.2376 - val_acc: 0.9225
Epoch 4/500
75032/75032 [==============================] - 54s 715us/sample - loss: 0.3348 - acc: 0.9250 - val_loss: 0.2043 - val_acc: 0.9387
Epoch 5/500
75032/75032 [==============================] - 54s 716us/sample - loss: 0.2944 - acc: 0.9354 - val_loss: 0.3416 - val_acc: 0.8745
Epoch 6/500
75032/75032 [==============================] - 54s 717us/sample - loss: 0.2722 - acc: 0.9381 - val_loss: 0.2492 - val_acc: 0.9121
Epoch 7/500
75032/75032 [==============================] - 55s 732us/sample - loss: 0.2538 - acc: 0

In [17]:
pred_y  = model.predict_classes(x_test)
print(classification_report(y_test,pred_y))


              precision    recall  f1-score   support

           0       1.00      0.91      0.95      8031
           1       0.29      0.95      0.44       306

    accuracy                           0.91      8337
   macro avg       0.64      0.93      0.70      8337
weighted avg       0.97      0.91      0.93      8337



In [2]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data"
x_test1 = pkl.load(open("x_test.p", "rb"))
y_test1 = pkl.load(open("y_test.p", "rb"))

x_test1.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data


(13362,)

In [3]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data"
x_test2 = pkl.load(open("x_test1911.p", "rb"))
y_test2 = pkl.load(open("y_test1911.p", "rb"))

x_test2.shape

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\data\test_data


(12362,)

In [4]:
x_test=[]
y_test=[]
smoking_event=0
non_smoking_event=0


for i, y in enumerate(y_test2):
  if x_test2[i].shape == (160, 4):
      if y in [1,2,3]:
        y_test.append(1)
        x_test.append(x_test2[i])
        smoking_event+=1
      elif y >= 20:
        y_test.append(0)
        x_test.append(x_test2[i])
        non_smoking_event+=1
        
x_test=np.array(x_test)
y_test=np.array(y_test)
x_test.shape

(10908, 160, 4)

In [6]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models"
best_model = tf.keras.models.load_model('model-21-0.9510.hdf5')
pred_y  = best_model.predict_classes(x_test)
print(classification_report(y_test,pred_y))
#1910 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
              precision    recall  f1-score   support

           0       0.99      0.92      0.95     10591
           1       0.53      0.91      0.67      1064

    accuracy                           0.92     11655
   macro avg       0.76      0.91      0.81     11655
weighted avg       0.95      0.92      0.93     11655



In [7]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models"
best_model = tf.keras.models.load_model('model-21-0.9510.hdf5')
pred_y  = best_model.predict_classes(x_test)
print(classification_report(y_test,pred_y))
#1911 test set

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models
              precision    recall  f1-score   support

           0       0.99      0.93      0.96     10188
           1       0.49      0.90      0.63       720

    accuracy                           0.93     10908
   macro avg       0.74      0.92      0.80     10908
weighted avg       0.96      0.93      0.94     10908



In [13]:
results_df.sort_values(by=['recall'], ascending = False)

,Layers_norm,Layers_filter,num_filters,num_nodes,initialization,val_acc,train_acc,percision,recall,f1
21,2.0,3.0,128.0,64.0,he_normal,0.946962,0.954060,0.421129,0.950980,0.583751
22,3.0,3.0,128.0,64.0,he_uniform,0.940832,0.944384,0.368957,0.947712,0.531136
7,3.0,3.0,64.0,64.0,he_normal,0.940432,0.944130,0.353733,0.944444,0.514693
2,3.0,2.0,64.0,64.0,he_uniform,0.958822,0.964255,0.460292,0.928105,0.615385
15,3.0,3.0,64.0,128.0,he_normal,0.963886,0.970639,0.522472,0.911765,0.664286
6,3.0,3.0,64.0,64.0,he_uniform,0.966285,0.970426,0.497297,0.901961,0.641115
14,3.0,3.0,64.0,128.0,he_uniform,0.964152,0.970919,0.512104,0.898693,0.652432
4,2.0,3.0,64.0,64.0,he_uniform,0.960954,0.967161,0.461149,0.892157,0.608018
8,2.0,2.0,64.0,128.0,he_uniform,0.961087,0.970013,0.478799,0.885621,0.621560
13,2.0,3.0,64.0,128.0,he_normal,0.972148,0.977796,0.555556,0.882353,0.681818


In [11]:
%cd "C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models"
results_df = pd.DataFrame()
class_weight = {0: 1, 1: 30}
#BatchSizes  = [32]
NumberOFilters = [64,128]
NumberOfNodes = [64,128]
FilterLayers = [2,3]
NormLayers = [2,3]
WeightInitialization = ['he_uniform', 'he_normal']
i = 0
early_stop=EarlyStopping(monitor='val_loss', min_delta=.0001, patience=20, verbose=0, mode='auto', baseline=None, restore_best_weights=False)
for num_filters in NumberOFilters:
    for num_nodes in NumberOfNodes:
        for num_layers_filter in FilterLayers:
            for num_layers_norm in NormLayers:
                for weight_intial in WeightInitialization:
                    model = Sequential()
                    model.add(Conv1D(filters=num_filters ,kernel_size = 2, activation='relu', kernel_initializer = weight_intial ,input_shape=(x_train.shape[1],x_train.shape[2])))
                    model.add(MaxPooling1D(pool_size=2))
                    model.add(Conv1D(filters=num_filters,kernel_initializer = weight_intial,kernel_size = 2 ,activation='relu'))
                    model.add(MaxPooling1D(pool_size=2))
                    if num_layers_filter == 3:
                        model.add(Conv1D(filters=num_filters,kernel_initializer = weight_intial,kernel_size = 2 ,activation='relu'))
                        model.add(MaxPooling1D(pool_size=2))
                    model.add(Flatten())
                    model.add(Dense(num_nodes,kernel_initializer = weight_intial, activation='elu'))
                    model.add(Dense(num_nodes/2,kernel_initializer =weight_intial, activation= 'elu'))
                    if num_layers_norm == 3:
                        model.add(Dense(num_nodes/4,kernel_initializer =weight_intial, activation= 'elu'))
                    model.add(Dense(1, kernel_initializer = weight_intial, activation='sigmoid'))
                    model.compile(optimizer='adam',loss='binary_crossentropy', metrics = ['accuracy'])
                    history = model.fit(x_train, y_train, batch_size=32, epochs=200, use_multiprocessing=True,validation_split=0.1, callbacks = [early_stop],class_weight = class_weight)
                    pred_y  = model.predict_classes(x_test)
                    train_loss, train_acc = model.evaluate(x_train, y_train, verbose=0)
                    historydf = pd.DataFrame.from_dict(history.history)
                    recall_score = sklearn.metrics.recall_score(y_test,pred_y)
                    model.save(f"model-{i}-{recall_score:.4f}.hdf5")
                    results_df.loc[i,'Layers_norm'] = num_layers_norm
                    results_df.loc[i,'Layers_filter'] = num_layers_filter
                    results_df.loc[i,'num_filters'] = num_filters
                    results_df.loc[i,'num_nodes'] = num_nodes 
                    results_df.loc[i,'initialization'] = weight_intial 
                    results_df.loc[i,'val_acc'] = history.history['val_acc'][-1]
                    results_df.loc[i,'train_acc'] = train_acc
                    results_df.loc[i,'percision'] = sklearn.metrics.precision_score(y_test,pred_y)
                    results_df.loc[i,'recall'] = sklearn.metrics.recall_score(y_test,pred_y)
                    results_df.loc[i,'f1'] = sklearn.metrics.f1_score(y_test,pred_y)
                    print(i)
                    i = i + 1

C:\Users\eric_\OneDrive\Documents\python workbooks\FreshAir\CNN model\models
Train on 67528 samples, validate on 7504 samples
Epoch 1/200
67528/67528 [==============================] - 21s 316us/sample - loss: 0.5980 - acc: 0.8541 - val_loss: 0.4643 - val_acc: 0.8410
Epoch 2/200
67528/67528 [==============================] - 21s 307us/sample - loss: 0.4390 - acc: 0.8997 - val_loss: 0.3821 - val_acc: 0.8811
Epoch 3/200
67528/67528 [==============================] - 28s 415us/sample - loss: 0.3857 - acc: 0.9150 - val_loss: 0.3372 - val_acc: 0.9015
Epoch 4/200
67528/67528 [==============================] - 73s 1ms/sample - loss: 0.3359 - acc: 0.9249 - val_loss: 0.3061 - val_acc: 0.9160
Epoch 5/200
67528/67528 [==============================] - 71s 1ms/sample - loss: 0.2990 - acc: 0.9340 - val_loss: 0.3742 - val_acc: 0.9014
Epoch 6/200
67528/67528 [==============================] - 68s 1ms/sample - loss: 0.2573 - acc: 0.9422 - val_loss: 0.3177 - val_acc: 0.9395
Epoch 7/200
67528/67528 [===

KeyboardInterrupt: 